In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import gzip
from subprocess import check_output
%matplotlib inline

In [ ]:
with open('../keys/tmdb_key.json') as fi:
    credentials = json.load(fi)

In [ ]:
api_key = credentials['api_key']

In [ ]:
tmdb_df = pd.read_json('../data/tmdb_movie_ids_12_16_2022.json.gz', lines=True, compression='gzip')
tmdb_df.info()

In [ ]:
display(tmdb_df)

In [ ]:
title_pop = tmdb_df[['id','original_title', 'popularity']]
display(title_pop)

In [ ]:
tmdb_ids = pd.read_excel(r'../data/tmdb_pulled_ids.xlsx', sheet_name='pulled_ids')
tmdb_ids = tmdb_ids.drop_duplicates(subset=['original_title'], keep=False)
print(tmdb_ids)

In [ ]:
movie_id = tmdb_ids['id'].tolist()

In [ ]:
def get_details(api_key, movie_id):
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US'
    
    res = requests.get(url)
    
    details_object = res.json()
    
    return details_object

In [ ]:
tmdb_details_info = []
for x in movie_id:
    tmdb_details_info.append(get_details(api_key, x))

In [ ]:
i = 0
title = []
release_date = []
popularity = []
budget = []
genres = []
revenue = []
production_company = []
while i < len(tmdb_details_info):
    popularity.append(tmdb_details_info[i]['popularity'])
    release_date.append(tmdb_details_info[i]['release_date'])
    title.append(tmdb_details_info[i]['original_title'])
    budget.append(tmdb_details_info[i]['budget'])
    try:
        genres.append(tmdb_details_info[i]['genres'][0]['name'])
    except IndexError:
        genres.append('null')
    try:
        production_company.append(tmdb_details_info[i]['production_companies'][0]['name'])
    except IndexError:
        production_company.append('null')
    revenue.append(tmdb_details_info[i]['revenue'])
    i = i+1

In [ ]:
tmdb_data = {'title':title, 'movie_popularity':popularity, 'movie_release_date':release_date, 'movie_budget':budget, 'movie_genre':genres, 'movie_revenue':revenue, 'movie_producers':production_company}
tmdb_df = pd.DataFrame(tmdb_data)
display(tmdb_df)